In [1]:
pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357294 sha256=69e582b32e2a4a4aa57fff36099f2a86b99f1e30d65ae2d0b539f15dd04595ff
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gauravduttakiit/million-song-dataset")

print("Path to dataset files:", path)

100%|██████████| 225M/225M [00:05<00:00, 46.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/gauravduttakiit/million-song-dataset/versions/1


In [3]:
# Import necessary libraries
import pandas as pd
import surprise
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import cross_validate

# Load the built-in MovieLens dataset ('ml-100k') from Surprise
# This dataset is for movie recommendations, but we'll use the same structure for music later
data = Dataset.load_builtin('ml-100k')

# Split dataset into training and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Use Singular Value Decomposition (SVD) algorithm for collaborative filtering
model = SVD()

# Train the model on the training set
model.fit(trainset)

# Make predictions on the test set
predictions = model.test(testset)

# Calculate accuracy metrics (e.g., RMSE)
accuracy.rmse(predictions)

# Cross-validate the model using 5-fold cross-validation
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Example: Predict whether a specific user will like a specific movie
user_id = '196'  # Replace with a real user_id from the MovieLens dataset
item_id = '302'  # Replace with a real item_id (movie_id) from the dataset
pred = model.predict(user_id, item_id)
print(f'Predicted rating for user {user_id} on item {item_id}: {pred.est}')

# Function to recommend items for a user (top 5 movies not yet rated by the user)
def recommend_items(user_id, model, n_recommendations=5):
    item_ids = trainset.all_items()
    user_items = [i for (i, _) in trainset.ur[trainset.to_inner_uid(user_id)]]
    recommendations = []
    for item_id in item_ids:
        if item_id not in user_items:  # Only recommend items the user hasn't interacted with
            pred = model.predict(user_id, str(trainset.to_raw_iid(item_id)))
            recommendations.append((item_id, pred.est))
    # Sort recommendations by predicted rating
    recommendations.sort(key=lambda x: x[1], reverse=True)
    return recommendations[:n_recommendations]

# Get top 5 movie recommendations for a user (user 196 in MovieLens dataset)
recommendations = recommend_items('196', model)
print("Top 5 recommendations for user 196:")
for item_id, score in recommendations:
    print(f'Item: {trainset.to_raw_iid(item_id)}, Predicted rating: {score}')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
RMSE: 0.9383
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9311  0.9431  0.9324  0.9383  0.9329  0.9355  0.0045  
MAE (testset)     0.7350  0.7447  0.7324  0.7388  0.7356  0.7373  0.0042  
Fit time          1.42    1.44    1.43    1.81    1.93    1.61    0.22    
Test time         0.12    0.29    0.18    0.21    0.24    0.21    0.06    
Predicted rating for user 196 on item 302: 4.676594004230606
Top 5 recommendations for user 196:
Item: 50, Predicted rating: 4.783768368593216
Item: 169, Predicted rating: 4.775462507174807
Item: 302, Predicted rating: 4.676594004230606
Item: 178, Predicted rating: 4.670202862754224
Item: 408, Predicted rating: 4.60125554